In [ ]:
!pip install pytorch-model-summary

In [ ]:
import os
from tqdm import tqdm
from pytorch_model_summary import summary
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
import cv2
import warnings
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from torchvision import models
import random
from sklearn.utils import shuffle
import numpy as np
warnings.simplefilter("ignore")

In [ ]:
!pip install opendatasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od

In [ ]:
dataset_url = 'https://www.kaggle.com/datasets/vbookshelf/v2-plant-seedlings-dataset'
od.download(dataset_url)

100%|██████████| 3.19G/3.19G [00:26<00:00, 131MB/s]


In [ ]:
def dataset_to_class_wise_csv(datafolder_path = "/content/v2-plant-seedlings-dataset"):
    list_of_dir = os.listdir(datafolder_path)
    list_of_dir.remove("nonsegmentedv2")
    dataset = list()
    for index,class_name in enumerate(list_of_dir):
        dir_name = os.path.join(datafolder_path , class_name)
        list_of_images = os.listdir(dir_name)
        for image_name in tqdm(list_of_images):
            image_path = os.path.join(dir_name,image_name)
            try:
                img = cv2.imread(image_path)
                img = cv2.resize(img, (256, 256))
                red , green , blue = cv2.split(img)
                pca = PCA(10)
                red_transformed = pca.fit_transform(red)
                ica = FastICA(n_components = 5,max_iter = 100)
                red_transformed = ica.fit_transform(red)
                dataset.append(dict(class_name = class_name,class_id = index,image_path = image_path))
            except:
                print("Currupt Image.")
    dataset = pd.DataFrame(dataset , columns = ["class_name","class_id","image_path"])
    return dataset
dataset = dataset_to_class_wise_csv()


In [ ]:
# train val and test split in 70% , 20% , 10%
x_train,x_test,y_train,y_test = train_test_split(dataset["image_path"],dataset["class_id"],test_size = 0.1 , stratify = dataset["class_id"])
x_train,x_val , y_train,y_val = train_test_split(x_train,y_train,test_size = 0.2 , stratify = y_train)

In [ ]:
def PCA_image_feature_extract(data):
    image_features = list()
    for image in tqdm(data):
        img = cv2.imread(image)
        img = cv2.resize(img, (200, 200))
        blue , green , red = cv2.split(img)
        pca = PCA(5)
        red_transformed = pca.fit_transform(red)
        green_transformed = pca.fit_transform(green)
        blue_transformed = pca.fit_transform(blue)
        img_compressed = random.choice([red_transformed,green_transformed,blue_transformed])
        img_compressed = img_compressed.flatten()
        image_features.append(img_compressed)
    image_features = np.stack(image_features , axis = 0)
    return image_features
def ICA_image_feature_extract(data):
    image_features = list()
    for image in tqdm(data):
        img = cv2.imread(image)
        img = cv2.resize(img, (200, 200))
        blue , green , red = cv2.split(img)
        ica = FastICA(n_components = 5,max_iter = 100)
        red_transformed = ica.fit_transform(red)
        green_transformed = ica.fit_transform(green)
        blue_transformed = ica.fit_transform(blue)
        img_compressed = random.choice([red_transformed,green_transformed,blue_transformed])
        img_compressed = img_compressed.flatten()
        image_features.append(img_compressed)
    image_features = np.stack(image_features , axis = 0)
    return image_features
pca_train_features,pca_test_features,pca_val_features = PCA_image_feature_extract(x_train),PCA_image_feature_extract(x_test),PCA_image_feature_extract(x_val)
ica_train_features,ica_test_features,ica_val_features = ICA_image_feature_extract(x_train),ICA_image_feature_extract(x_test),ICA_image_feature_extract(x_val)

In [ ]:
import pickle
with open("../input/pca-features/pca_features.pkl","rb") as file_obj:
    pca_train_features,pca_test_features,pca_val_features = pickle.load(file_obj)
    file_obj.close()
with open("../input/ica-features/ica_features.pkl","rb") as file_obj:
    ica_train_features,ica_test_features,ica_val_features = pickle.load(file_obj)
    file_obj.close()

In [ ]:
class CustomDataset(Dataset):
    def __init__(self,x,y,transforms,pca_features,ica_features):
        super(CustomDataset,self).__init__()
        self.x = list(x)
        self.y = list(y)
        self.pca_features = pca_features
        self.ica_features = ica_features
        self.transforms = transforms
    def __len__(self):
        return len(self.x)
    def __getitem__(self,index):
        image = Image.open(self.x[index]).convert("RGB")
        if self.transforms != None:
            image = self.transforms(image)
        return image,torch.tensor(self.y[index],dtype = torch.long),torch.tensor(self.pca_features[index],dtype = torch.float32),torch.tensor(self.ica_features[index],dtype = torch.float32)
def get_dataloader(x_train,y_train,x_test,y_test,x_val,y_val,
                   pca_train_features,pca_test_features,pca_val_features,
                   ica_train_features,ica_test_features,ica_val_features,batch_size):
    transform = transforms.Compose([transforms.ToTensor(),
                                   transforms.Resize((256,256)),
                                  transforms.Normalize(mean = (0.485, 0.456, 0.406),std = (0.229, 0.224, 0.225))
                                   ])
    train_dataset = CustomDataset(x_train,y_train,transform,pca_train_features,ica_train_features)
    val_dataset = CustomDataset(x_val,y_val,transform,pca_val_features,ica_val_features)
    test_dataset = CustomDataset(x_test , y_test,transform,pca_test_features,ica_test_features)
    train_dataloader = DataLoader(dataset = train_dataset , batch_size = batch_size , shuffle = False)
    test_dataloader = DataLoader(dataset = test_dataset , batch_size = batch_size , shuffle = False)
    val_dataloader = DataLoader(dataset = val_dataset , batch_size = batch_size , shuffle = False)
    return train_dataloader , test_dataloader , val_dataloader
batch_size = 48
train_dataloader , test_dataloader , val_dataloader = get_dataloader(x_train,y_train,x_test,y_test,x_val ,y_val,
                                                                     pca_train_features,pca_test_features,pca_val_features,
                                                                     ica_train_features,ica_test_features,ica_val_features,batch_size)

In [ ]:
class Model1(nn.Module):
    def __init__(self,num_classes):
        super(Model1 , self).__init__()
        self.num_classes = num_classes
        self.resnet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
        self.classifier = nn.Sequential(nn.Linear(in_features = 3000,out_features = 256),
                                       nn.ReLU(),
                                       nn.BatchNorm1d(256),
                                       nn.Dropout(p = 0.5),
                                       nn.Linear(in_features = 256 , out_features = self.num_classes),
                                       nn.Softmax())
    def forward(self,x,ml_features1,ml_features2):
        resnet_output = self.resnet_model(x)
        concatenated_output = torch.cat((resnet_output,ml_features1,ml_features2),axis = 1) # (batch_size , 3000)
        final_output = self.classifier(concatenated_output)
        return final_output
class Model2(nn.Module):
    def __init__(self,num_classes):
        super(Model2 , self).__init__()
        self.num_classes = num_classes
        self.googlenet_model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
        self.classifier = nn.Sequential(nn.Linear(in_features = 3000,out_features = 256),
                                       nn.ReLU(),
                                       nn.BatchNorm1d(256),
                                       nn.Dropout(p = 0.5),
                                       nn.Linear(in_features = 256 , out_features = self.num_classes),
                                       nn.Softmax())
    def forward(self,x,ml_features1,ml_features2):
        google_net_output = self.googlenet_model(x)
        concatenated_output = torch.cat((google_net_output,ml_features1,ml_features2),axis = 1) # (batch_size , 3000)
        final_output = self.classifier(concatenated_output)
        return final_output
class Model3(nn.Module):
    def __init__(self,num_classes):
        super(Model3 , self).__init__()
        self.num_classes = num_classes
        self.inception_model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
        self.inception_model = nn.Sequential(*(list(self.inception_model.children())[:-1]))
        self.inception_last_layer = nn.Linear(in_features = 2048 , out_features = 1000)
        self.classifier = nn.Sequential(nn.Linear(in_features = 3000,out_features = 512),
                                       nn.ReLU(),
                                       nn.BatchNorm1d(512),
                                       nn.Dropout(p = 0.5),
                                       nn.Linear(in_features = 512 , out_features = self.num_classes),
                                       nn.Softmax())
    def forward(self,x,ml_features1,ml_features2):
        inception_output = self.inception_model[0](x)
        for layer_no in range(1,20):
            if layer_no != 15:
                inception_output = self.inception_model[layer_no](inception_output)
        inception_output = inception_output.view(inception_output.size(0),-1) #(batch_size , 2048)
        inception_output = self.inception_last_layer(inception_output)
        concatenated_output = torch.cat((inception_output,ml_features1,ml_features2),axis = 1) # (batch_size , 3000)
        final_output = self.classifier(concatenated_output)
        return final_output



In [ ]:
print(summary(Model1(num_classes=12), torch.zeros((1, 3, 256, 256)), torch.zeros((1,1000)),torch.zeros((1,1000)) ,show_input=True))
print(summary(Model2(num_classes=12), torch.zeros((1, 3, 256, 256)), torch.zeros((1,1000)),torch.zeros((1,1000)) ,show_input=True))
print(summary(Model3(num_classes=12), torch.zeros((1, 3, 256, 256)), torch.zeros((1,1000)),torch.zeros((1,1000)) ,show_input=True))

In [ ]:
class Train_DL_Model(object):
    def __init__(self,train_dataloader,test_dataloader,validation_dataloader,
                 model,n_epochs,n_classes,lr = 0.00001):
        self.train_dataloader = train_dataloader
        self.test_dataloader = test_dataloader
        self.validation_dataloader = validation_dataloader
        self.n_classes = n_classes
        self.n_epochs = n_epochs
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model(self.n_classes).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters() , lr = self.lr)
        self.criterion = nn.CrossEntropyLoss()
        self.train_prediction = []
        self.validation_prediction = []
        self.test_prediction = []
        self.train_original = []
        self.test_original = []
        self.val_original = []
    def train_model(self):
        print("Training Start...")
        num_batch = len(self.train_dataloader)
        for epoch in range(self.n_epochs):
            print("| ********* Epoch : {} ********** |".format(epoch+1))
            for index,(image,label,pca_features,ica_features) in enumerate(self.train_dataloader):
                image = image.to(self.device)
                label = label.to(self.device)
                ica_features = ica_features.to(self.device)
                pca_features = pca_features.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(image,ica_features,pca_features)
                loss = self.criterion(output , label)
                loss.backward()
                self.optimizer.step()
                if (index) % 10 == 0:
                    print("| Batch : {0}/{1} |".format(index, num_batch))
        print("Training End .....")
    def get_test_validation_prediction(self,dataloader):
        num_batch = len(dataloader)
        with torch.no_grad():
            total_prediction = []
            total_original = []
            probability= []
            for index,(image,label,pca_features,ica_features) in enumerate(dataloader):
                image = image.to(self.device)
                ica_features = ica_features.to(self.device)
                pca_features = pca_features.to(self.device)
                output = self.model(image,ica_features,pca_features)
                prediction = torch.argmax(output , axis = 1).cpu().tolist()
                total_original.extend(label.tolist())
                total_prediction.extend(prediction)
                probability.extend(output.cpu().tolist())
                if (index) % 10 == 0:
                    print("| Batch : {0}/{1} |".format(index, num_batch))
        return total_prediction,total_original,probability
    def test_validation(self):
        print("Training Prediction Start ")
        pred , original,train_probability = self.get_test_validation_prediction(self.train_dataloader)
        self.train_prediction.extend(pred)
        self.train_original.extend(original)
        print("Training Prediction End ")
        print("Validation Prediction Start ")
        pred , original, val_probability= self.get_test_validation_prediction(self.validation_dataloader)
        self.validation_prediction.extend(pred)
        self.val_original.extend(original)
        print("Validation Prediction End ")
        print("Testing Prediction Start ")
        pred , original ,test_probability= self.get_test_validation_prediction(self.test_dataloader)
        self.test_prediction.extend(pred)
        self.test_original.extend(original)
        print("Validation Prediction End ")
        return (self.train_prediction,self.validation_prediction , self.test_prediction),(self.train_original,self.val_original,self.test_original),(train_probability,val_probability,test_probability)


In [ ]:
from sklearn.metrics import accuracy_score , f1_score,recall_score,precision_score,classification_report
import pickle
def calulate_result(original_data,DL):
    accuracy = dict(train = accuracy_score(original_data[0],DL[0])*100,
                   val = accuracy_score(original_data[1],DL[1])*100,
                   test = accuracy_score(original_data[2],DL[2])*100)
    print("accuracy", accuracy)
    recall = dict(train = recall_score(original_data[0],DL[0],average = "weighted")*100,
                   val = recall_score(original_data[1],DL[1],average = "weighted")*100,
                   test = recall_score(original_data[2],DL[2],average = "weighted")*100)
    print("recall", recall)
    precision = dict(train = precision_score(original_data[0],DL[0],average = "weighted")*100,
                   val = precision_score(original_data[1],DL[1],average = "weighted")*100,
                   test = precision_score(original_data[2],DL[2],average = "weighted")*100)
    print("precision", precision)
    f1 = dict(train = f1_score(original_data[0],DL[0],average = "weighted")*100,
                   val = f1_score(original_data[1],DL[1],average = "weighted")*100,
                   test = f1_score(original_data[2],DL[2],average = "weighted")*100)
    print("f1", f1)
    report = dict(train = classification_report(original_data[0],DL[0],output_dict = True),
                   val = classification_report(original_data[1],DL[1],output_dict = True),
                   test = classification_report(original_data[2],DL[2],output_dict = True))

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import precision_recall_curve, roc_curve
def plot_curve(original_data, DL, title):
  y_7 = pd.Series(original_data)
  new_labels = {7:1, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_7 = y_7.map(new_labels)
  y_7 = y_7.to_numpy()

  y_5 = pd.Series(original_data)
  new_labels = {7:0, 5:1, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_5 = y_5.map(new_labels)
  y_5 = y_5.to_numpy()

  y_8 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:1, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_8 = y_8.map(new_labels)
  y_8 = y_8.to_numpy()

  y_6 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:0, 6:1, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_6 = y_6.map(new_labels)
  y_6 = y_6.to_numpy()

  y_11 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:1, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_11 = y_11.map(new_labels)
  y_11 = y_11.to_numpy()

  y_1 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:1, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_1 = y_1.map(new_labels)
  y_1 = y_1.to_numpy()

  y_4 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:1, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_4 = y_4.map(new_labels)
  y_4 = y_4.to_numpy()

  y_9 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:1, 2:0, 10:0, 3:0, 0:0}
  y_9 = y_9.map(new_labels)
  y_9 = y_9.to_numpy()

  y_2 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:1, 10:0, 3:0, 0:0}
  y_2 = y_2.map(new_labels)
  y_2 = y_2.to_numpy()

  y_10 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:1, 3:0, 0:0}
  y_10 = y_10.map(new_labels)
  y_10 = y_10.to_numpy()

  y_3 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:1, 0:0}
  y_3 = y_3.map(new_labels)
  y_3 = y_3.to_numpy() 

  y_0 = pd.Series(original_data)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:1}
  y_0 = y_0.map(new_labels)
  y_0 = y_0.to_numpy()

  y_7_pred = pd.Series(DL)
  new_labels = {7:1, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_7_pred = y_7_pred.map(new_labels)
  y_7_pred = y_7_pred.to_numpy()

  y_5_pred = pd.Series(DL)
  new_labels = {7:0, 5:1, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_5_pred = y_5_pred.map(new_labels)
  y_5_pred = y_5_pred.to_numpy()

  y_8_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:1, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_8_pred = y_8_pred.map(new_labels)
  y_8_pred = y_8_pred.to_numpy()

  y_6_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:0, 6:1, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_6_pred = y_6_pred.map(new_labels)
  y_6_pred = y_6_pred.to_numpy()

  y_11_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:1, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_11_pred = y_11_pred.map(new_labels)
  y_11_pred = y_11_pred.to_numpy()

  y_1_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:1, 4:0, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_1_pred = y_1_pred.map(new_labels)
  y_1_pred = y_1_pred.to_numpy()

  y_4_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:1, 9:0, 2:0, 10:0, 3:0, 0:0}
  y_4_pred = y_4_pred.map(new_labels)
  y_4_pred = y_4_pred.to_numpy()

  y_9_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:1, 2:0, 10:0, 3:0, 0:0}
  y_9_pred = y_9_pred.map(new_labels)
  y_9_pred = y_9_pred.to_numpy()

  y_2_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:1, 10:0, 3:0, 0:0}
  y_2_pred = y_2_pred.map(new_labels)
  y_2_pred = y_2_pred.to_numpy()

  y_10_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:1, 3:0, 0:0}
  y_10_pred = y_10_pred.map(new_labels)
  y_10_pred = y_10_pred.to_numpy()

  y_3_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:1, 0:0}
  y_3_pred = y_3_pred.map(new_labels)
  y_3_pred = y_3_pred.to_numpy() 

  y_0_pred = pd.Series(DL)
  new_labels = {7:0, 5:0, 8:0, 6:0, 11:0, 1:0, 4:0, 9:0, 2:0, 10:0, 3:0, 0:1}
  y_0_pred = y_0_pred.map(new_labels)
  y_0_pred = y_0_pred.to_numpy()

  precision_c0, recall_c0, _ = precision_recall_curve(y_0, y_0_pred)
  precision_c1, recall_c1, _ = precision_recall_curve(y_1, y_1_pred)
  precision_c2, recall_c2, _ = precision_recall_curve(y_2, y_2_pred)
  precision_c3, recall_c3, _ = precision_recall_curve(y_3, y_3_pred)
  precision_c4, recall_c4, _ = precision_recall_curve(y_4, y_4_pred)
  precision_c5, recall_c5, _ = precision_recall_curve(y_5, y_5_pred)
  precision_c6, recall_c6, _ = precision_recall_curve(y_6, y_6_pred)
  precision_c7, recall_c7, _ = precision_recall_curve(y_7, y_7_pred)
  precision_c8, recall_c8, _ = precision_recall_curve(y_8, y_8_pred)
  precision_c9, recall_c9, _ = precision_recall_curve(y_9, y_9_pred)
  precision_c10, recall_c10, _ = precision_recall_curve(y_10, y_10_pred)
  precision_c11, recall_c11, _ = precision_recall_curve(y_11, y_11_pred)
  
  fpr0, tpr0, _ = roc_curve(y_0, y_0_pred)
  fpr1, tpr1, _ = roc_curve(y_1, y_1_pred)
  fpr2, tpr2, _ = roc_curve(y_2, y_2_pred)
  fpr3, tpr3, _ = roc_curve(y_3, y_3_pred)
  fpr4, tpr4, _ = roc_curve(y_4, y_4_pred)
  fpr5, tpr5, _ = roc_curve(y_5, y_5_pred)
  fpr6, tpr6, _ = roc_curve(y_6, y_6_pred)
  fpr7, tpr7, _ = roc_curve(y_7, y_7_pred)
  fpr8, tpr8, _ = roc_curve(y_8, y_8_pred)
  fpr9, tpr9, _ = roc_curve(y_9, y_9_pred)
  fpr10, tpr10, _ = roc_curve(y_10, y_10_pred)
  fpr11, tpr11, _ = roc_curve(y_11, y_11_pred)
  
  #plot precision-recall curve
  plt.plot(precision_c0, recall_c0)
  plt.plot(precision_c1, recall_c1)
  plt.plot(precision_c2, recall_c2)
  plt.plot(precision_c3, recall_c3)
  plt.plot(precision_c4, recall_c4)
  plt.plot(precision_c5, recall_c5)
  plt.plot(precision_c6, recall_c6)
  plt.plot(precision_c7, recall_c7)
  plt.plot(precision_c8, recall_c8)
  plt.plot(precision_c9, recall_c9)
  plt.plot(precision_c10, recall_c10)
  plt.plot(precision_c11, recall_c11)
  plt.legend(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"], loc ="lower right")
  plt.title(title)
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.xlabel('Precision')
  plt.ylabel('Recall')
  plt.show()
  plt.savefig("PR-AUC-merged-BiGRUx3.png", dpi = 200)

  # Plot ROC AUC
  plt.plot(fpr0, tpr0)
  plt.plot(fpr1, tpr1)
  plt.plot(fpr2, tpr2)
  plt.plot(fpr3, tpr3)
  plt.plot(fpr4, tpr4)
  plt.plot(fpr5, tpr5)
  plt.plot(fpr6, tpr6)
  plt.plot(fpr7, tpr7)
  plt.plot(fpr8, tpr8)
  plt.plot(fpr9, tpr9)
  plt.plot(fpr10, tpr10)
  plt.plot(fpr11, tpr11)
  plt.legend(["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"], loc ="lower right")
  plt.title(title)
  plt.plot([0, 1], [0, 1],'r--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.ylabel('True Positive Rate')
  plt.xlabel('False Positive Rate')
  plt.show()
  plt.savefig("ROC-AUC-merged-BiGRUx3.png", dpi = 200)

In [ ]:
# Train Model 1
n_epochs = 1
model1 = Train_DL_Model(train_dataloader,test_dataloader,val_dataloader,Model1,n_epochs,12)
model1.train_model()
model1_pred,model1_original,pred_probability1 = model1.test_validation()

In [ ]:
calulate_result(model1_original,model1_pred)

accuracy {'train': 9.979939819458375, 'val': 8.826479438314944, 'test': 11.191335740072201}
recall {'train': 9.979939819458375, 'val': 8.826479438314944, 'test': 11.191335740072201}
precision {'train': 10.913101434169551, 'val': 10.031375420901007, 'test': 11.92452856381401}
f1 {'train': 10.244326663489261, 'val': 9.15710242160189, 'test': 11.28641164517959}


In [ ]:
plot_curve(model1_original[0], model1_pred[0], 'Train')
plot_curve(model1_original[1], model1_pred[1], 'Valiadation')
plot_curve(model1_original[2], model1_pred[2], 'Test')

In [ ]:
# Train Model 2
n_epochs = 1
model2 = Train_DL_Model(train_dataloader,test_dataloader,val_dataloader,Model2,n_epochs,12)
model2.train_model()
model2_pred,model2_original,pred_probability2 = model2.test_validation()

In [ ]:
calulate_result(model2_original,model2_pred)

accuracy {'train': 9.804413239719159, 'val': 7.823470411233702, 'test': 8.303249097472925}
recall {'train': 9.804413239719159, 'val': 7.823470411233702, 'test': 8.303249097472925}
precision {'train': 10.324424651992063, 'val': 8.15242953492156, 'test': 8.910496540495181}
f1 {'train': 9.697134047202235, 'val': 7.7674269965346046, 'test': 8.298553614452116}


In [ ]:
plot_curve(model2_original[0], model2_pred[0], 'Train')
plot_curve(model2_original[1], model2_pred[1], 'Valiadation')
plot_curve(model2_original[2], model2_pred[2], 'Test')

In [ ]:
# Train Model 3
n_epochs = 1
model3 = Train_DL_Model(train_dataloader,test_dataloader,val_dataloader,Model3,n_epochs,12)
model3.train_model()
model3_pred,model3_original,pred_probability3 = model3.test_validation()

In [ ]:
calulate_result(model3_original,model3_pred)

accuracy {'train': 9.804413239719159, 'val': 9.42828485456369, 'test': 9.025270758122744}
recall {'train': 9.804413239719159, 'val': 9.42828485456369, 'test': 9.025270758122744}
precision {'train': 10.91769421415426, 'val': 10.986131477757672, 'test': 10.28557426765606}
f1 {'train': 10.084656929224273, 'val': 9.840005177516067, 'test': 9.27162931057184}


In [ ]:
plot_curve(model3_original[0], model3_pred[0], 'Train')
plot_curve(model3_original[1], model3_pred[1], 'Valiadation')
plot_curve(model3_original[2], model3_pred[2], 'Test')

In [ ]:
from collections import Counter
train_prob = []
for model1,model2,model3,p1,p2,p3 in zip(model1_pred[0],model2_pred[0],model3_pred[0],pred_probability1[0],pred_probability2[0],pred_probability3[0]):
    counter = Counter([model1,model2,model3])
    counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)[0][0]
    if counter == model1:
        train_prob.append(p1)
    elif counter == model2:
        train_prob.append(p2)
    elif counter == model3:
        train_prob.append(p3)
# validation prediction
val_prob = []
for model1,model2,model3,p1,p2,p3 in zip(model1_pred[1],model2_pred[1],model3_pred[1],pred_probability1[1],pred_probability2[1],pred_probability3[1]):
    counter = Counter([model1,model2,model3])
    counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)[0][0]
    if counter == model1:
        val_prob.append(p1)
    elif counter == model2:
        val_prob.append(p2)
    elif counter == model3:
        val_prob.append(p3)
# test prediction
test_prob = []
for model1,model2,model3,p1,p2,p3 in zip(model1_pred[2],model2_pred[2],model3_pred[2],pred_probability1[2],pred_probability2[2],pred_probability3[2]):
    counter = Counter([model1,model2,model3])
    counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)[0][0]
    if counter == model1:
        test_prob.append(p1)
    elif counter == model2:
        test_prob.append(p2)
    elif counter == model3:
        test_prob.append(p3)

In [ ]:
from collections import Counter
from sklearn.metrics import accuracy_score , f1_score,recall_score,precision_score,classification_report
import pickle
def calulate_result(original_data,Model1,Model2,Model3,pred_probability):
    # train_prediction
    train_prediction = []
    for model1,model2,model3 in zip(Model1[0],Model2[0],Model3[0]):
        counter = Counter([model1,model2,model3])
        counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)[0][0]
        train_prediction.append(counter)
    # validation prediction
    val_prediction = []
    for model1,model2,model3 in zip(Model1[1],Model2[1],Model3[1]):
        counter = Counter([model1,model2,model3])
        counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)[0][0]
        val_prediction.append(counter)
    # test prediction
    test_prediction = []
    for model1,model2,model3 in zip(Model1[2],Model2[2],Model3[2]):
        counter = Counter([model1,model2,model3])
        counter = sorted(counter.items(), key=lambda x: x[1], reverse=True)[0][0]
        test_prediction.append(counter)
    accuracy = dict(train = accuracy_score(original_data[0],train_prediction)*100,
                   val = accuracy_score(original_data[1],val_prediction)*100,
                   test = accuracy_score(original_data[2],test_prediction)*100)
    print("accuracy", accuracy)
    recall = dict(train = recall_score(original_data[0],train_prediction,average = "weighted")*100,
                   val = recall_score(original_data[1],val_prediction,average = "weighted")*100,
                   test = recall_score(original_data[2],test_prediction,average = "weighted")*100)
    print("recall", recall)
    precision = dict(train = precision_score(original_data[0],train_prediction,average = "weighted")*100,
                   val = precision_score(original_data[1],val_prediction,average = "weighted")*100,
                   test = precision_score(original_data[2],test_prediction,average = "weighted")*100)
    print("precision", precision)
    f1 = dict(train = f1_score(original_data[0],train_prediction,average = "weighted")*100,
                   val = f1_score(original_data[1],val_prediction,average = "weighted")*100,
                   test = f1_score(original_data[2],test_prediction,average = "weighted")*100)
    print("f1", f1)
    report = dict(train = classification_report(original_data[0],train_prediction,output_dict = True),
                   val = classification_report(original_data[1],val_prediction,output_dict = True),
                   test = classification_report(original_data[2],test_prediction,output_dict = True))
    with open("Architecure_9_result.pkl" , "wb") as file_obj:
        data = dict(accuracy = accuracy,recall = recall,precision = precision,f1 = f1,report = report,name = "Architecure_9_result",
                   original_data = original_data , model_prediction = [train_prediction,val_prediction,test_prediction],probability = pred_probability)
        pickle.dump(data , file_obj)
calulate_result(model1_original,model1_pred,model2_pred,model3_pred,(train_prob,val_prob,test_prob))

In [ ]:
train_probability , val_probability , test_probability =np.array(train_prob),np.array(val_prob),np.array(test_prob)
y_train,y_val,y_test = np.array(model3_original[0]),np.array(model3_original[1]),np.array(model3_original[2])

In [ ]:
from sklearn.metrics import roc_curve, auc
from itertools import cycle
import matplotlib.pyplot as plt

# TRAIN ROC

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
lw=2
n_classes = 12
one_hot = np.zeros((y_train.shape[0],n_classes))
for index,label in enumerate(y_train):
    one_hot[index][label] = 1
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(one_hot[:, i], train_probability[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],  lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multi-class data')
plt.legend(loc="lower right")
plt.show()

# VAL ROC

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
lw=2
n_classes = 12
one_hot = np.zeros((y_val.shape[0],n_classes))
for index,label in enumerate(y_val):
    one_hot[index][label] = 1
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(one_hot[:, i], val_probability[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],  lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multi-class data')
plt.legend(loc="lower right")
plt.show()

# TEST ROC

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()
lw=2
n_classes = 12
one_hot = np.zeros((y_test.shape[0],n_classes))
for index,label in enumerate(y_test):
    one_hot[index][label] = 1
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(one_hot[:, i], test_probability[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i],  lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for multi-class data')
plt.legend(loc="lower right")
plt.show()